Data: 15/05/23

Descrição: Código para unir os datasets do Kaggle e salvar o dataset final no GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Download data from Kaggle

In [ ]:
# Carregando arquivo kaggle.json para podermos baixar datasets do Kaggle

from google.colab import files
files.upload() #this will prompt you to upload the kaggle.json

In [ ]:
# Instalando biblioteca do kaggle
!pip install -q kaggle

In [ ]:
# Checando diretório atual
!pwd

/content


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Indo para o diretório do projeto no Drive
%cd /content/drive/Shareddrives/IA901 - Projeto Final/Datasets/Raw

/content/drive/Shareddrives/IA901 - Projeto Final/Datasets/Raw


In [ ]:
# Baixando datasets do kaggle

#!kaggle datasets download -d your_dataset_name
!kaggle datasets download -d andrewmvd/cancer-inst-segmentation-and-classification
!kaggle datasets download -d andrewmvd/cancer-instance-segmentation-and-classification-2
!kaggle datasets download -d andrewmvd/cancer-instance-segmentation-and-classification-3

 99% 1.30G/1.32G [00:15<00:00, 126MB/s]
100% 1.32G/1.32G [00:15<00:00, 93.4MB/s]
 99% 691M/697M [00:12<00:00, 78.9MB/s]
100% 697M/697M [00:12<00:00, 60.6MB/s]
 99% 753M/759M [00:17<00:00, 87.5MB/s]
100% 759M/759M [00:17<00:00, 44.3MB/s]


In [ ]:
# Deszipando os arquivos

# Part 1:
!mkdir -p Part1
!unzip cancer-inst-segmentation-and-classification.zip -d Part1

!mkdir -p Part2
!unzip cancer-instance-segmentation-and-classification-2.zip -d Part2

!mkdir -p Part3
!unzip cancer-instance-segmentation-and-classification-3.zip -d Part3

Archive:  cancer-instance-segmentation-and-classification-2.zip
  inflating: Part2/Author Notes/README.md  
  inflating: Part2/Author Notes/README_data.md  
  inflating: Part2/Author Notes/by-nc-sa.md  
  inflating: Part2/Images/images.npy  
  inflating: Part2/Images/types.npy  
  inflating: Part2/Masks/masks.npy   
Archive:  cancer-instance-segmentation-and-classification-3.zip
  inflating: Part3/Author Notes/README.md  
  inflating: Part3/Author Notes/README_data.md  
  inflating: Part3/Author Notes/by-nc-sa.md  
  inflating: Part3/Images/images.npy  
  inflating: Part3/Images/types.npy  
  inflating: Part3/Masks/masks.npy   


# Juntando datasets

In [ ]:
import numpy as np
from PIL import Image
import pandas as pd

PATH = '/content/drive/Shareddrives/IA901 - Projeto Final/Datasets'

## Images

In [ ]:
img_array1 = np.load(PATH+'/Raw/Part1/Part 1/Images/images.npy')

%cd /content/drive/Shareddrives/IA901 - Projeto Final/Datasets/Interim

!mkdir -p Images
print('N images in dataset 1', img_array1.shape[0])

for i in range(0, img_array1.shape[0]):
  im = Image.fromarray(img_array1[i].astype('uint8'))
  im.save("Images/Image_{}.png".format(i))

In [ ]:
%cd 
#img_array2 = np.load(PATH+'/Raw/Part2/Images/images.npy')

print('N images in dataset 2', img_array2.shape[0])

Imagecont = 2656
for i in range(0, img_array2.shape[0]):
  im = Image.fromarray(img_array2[i].astype('uint8'))
  im.save(PATH+"/Interim/Images/Image_{}.png".format(Imagecont))
  Imagecont = Imagecont + 1

print('Imagecont: ', Imagecont)

/root
N images in dataset 2 2523
Imagecont:  5179


In [ ]:
img_array3 = np.load(PATH+'/Raw/Part3/Images/images.npy')
print('N images in dataset 3', img_array3.shape[0])

Imagecont = 5179
for i in range(0, img_array3.shape[0]):
  im = Image.fromarray(img_array3[i].astype('uint8'))
  im.save(PATH+"/Interim/Images/Image_{}.png".format(Imagecont))
  Imagecont = Imagecont + 1

print('Imagecont: ', Imagecont)

N images in dataset 3 2722
Imagecont:  7901


## Masks

In [ ]:
# Masks dataset 1
img_masks1 = np.load(PATH+'/Raw/Part1/Part 1/Masks/masks.npy')

# Discovering the class of each image
ClassSTD = []
for i in range(0, img_masks1.shape[0]):
  b = img_masks1[i, :, :, 0] # neoplasic channel
  std = np.std(b)
  ClassSTD.append(std)

ClassDF = pd.DataFrame(ClassSTD, columns=['Class'])
ClassDF['Class'][ClassDF.Class>0.0] = 1 # Criando classes
ClassDF.to_csv(PATH+"/Raw/Classes_Part1.csv", index=False, encoding='utf-8')

In [ ]:
# Masks dataset 2

ClassSTD = []
img_masks2 = np.load(PATH+'/Raw/Part2/Masks/masks.npy')
for i in range(0, img_masks2.shape[0]):
  b = img_masks2[i, :, :, 0] # neoplasic channel
  std = np.std(b)
  ClassSTD.append(std)

ClassDF = pd.DataFrame(ClassSTD, columns=['Class'])
ClassDF['Class'][ClassDF.Class>0.0] = 1 # Criando classes
ClassDF.to_csv(PATH+"/Raw/Classes_Part2.csv", index=False, encoding='utf-8')

In [ ]:
# Masks dataset 3

ClassSTD = []
img_masks3 = np.load(PATH+'/Raw/Part3/Masks/masks.npy')
for i in range(0, img_masks3.shape[0]):
  b = img_masks3[i, :, :, 0] # neoplasic channel
  std = np.std(b)
  ClassSTD.append(std)

ClassDF = pd.DataFrame(ClassSTD, columns=['Class'])
ClassDF['Class'][ClassDF.Class>0.0] = 1 # Criando classes
ClassDF.to_csv(PATH+"/Raw/Classes_Part3.csv", index=False, encoding='utf-8')


In [ ]:
# Join Class dataframes
Class1 = pd.read_csv(PATH+"/Raw/Classes_Part1.csv")
Class2 = pd.read_csv(PATH+"/Raw/Classes_Part2.csv")
Class3 = pd.read_csv(PATH+"/Raw/Classes_Part3.csv")

DataFrameClass = pd.concat([Class1, Class2, Class3], axis=0, ignore_index=True)
#DataFrameClass.to_csv(PATH+"/Interim/Class.npy", index=False, encoding='utf-8')

## Types

In [ ]:
# Types:

Types = []
Types1 = np.load(PATH+'/Raw/Part1/Part 1/Images/types.npy')
Types2 = np.load(PATH+'/Raw/Part2/Images/types.npy')
Types3 = np.load(PATH+'/Raw/Part3/Images/types.npy')

Types.append(Types1) # dataset1
Types.append(Types2) # dataset2
Types.append(Types3) # dataset3
TypesData = np.concatenate(Types)

#np.save(PATH+'/Interim/Types.npy', TypesData)
Types = pd.DataFrame(TypesData, columns=['Types'])

# Juntando Class e Types em um mesmo .csv
DataFrameClassTypes = pd.concat([Types, DataFrameClass], axis=1)
DataFrameClassTypes.to_csv(PATH+"/Interim/ClassAndTypes.csv", index=True, encoding='utf-8')

In [ ]:
DataFrameClassTypes

,Types,Class
0,Breast,1.0
1,Breast,1.0
2,Breast,1.0
3,Breast,1.0
4,Breast,1.0
...,...,...
7896,Colon,1.0
7897,Colon,1.0
7898,Colon,0.0
7899,Colon,0.0
